In [3]:
from fuzzywuzzy import fuzz
import random
import string

/Users/zacharygarcia/anaconda2/lib/python2.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


First, a simple function to create N random strings and select the one that is closest to the target string

In [7]:
def shotgun_approach(population_size, target_string):
    
    string_length = len(target_string)
    
    def new_member(length):
        letters = string.ascii_lowercase
        member = ''.join(random.choice(letters) for i in range(length))
        return member
    
    members = []
    
    for i in range(population_size):
        members.append(new_member(string_length))
        
    best_string = members[0]
    best_score = fuzz.ratio(members[0], target_string)
    
    for i in members:
        score = fuzz.ratio(i, target_string)
        if score > best_score:
            best_score = score
            best_string = i
    
    return best_string, best_score
    

In [10]:
shotgun_approach(10000, 'potato')

('ptftco', 67)

Now, let's try for the real thing!

# Components of the genetic algorithm:
[source](https://towardsdatascience.com/introduction-to-genetic-algorithms-including-example-code-e396e98d8bf3)

## 1: Initial population
This is an initial set of possible solutions to your problem, or members of the population. Each is unique, and the population has a fixed size. this means that as new members are created, less fit ones 'die'.

## 2: Fitness function
A function that evaluates the fitness of a member based on some criteria, eg. how close a string is to your target string. This should output a continuous score rather than a binary value.

[manatee, cat, parakeet, cheetah] => [1, 3, 7, 9]

## 3: Selection
The process of running the fitness function and choosing the fittest individuals (survival of the fittest)

[1, 3, 7, 9] => [9, 7]

## 4: Crossover
Analogous to recombination in biology - this is the random selection of a crossover point between two individuals, 

AAAAA + BBBBB = [AABBB, BBAAA]

## 5: Mutation
In newly formed offspring, some 'genes' can be subject to mutation with a low random probability. This serves to maintain diversity and prevent premature convergence.

AABBB => ABABB

### Termination
The algorithm ends when the population has **converged**, meaning subsequent generations do not produce significantly different offspring.

In [197]:

target = 'abigail'
word_length = len(target)
population_size = 100



class Member:
    def __init__(self, length):
        letters = string.ascii_lowercase
        self.ztext = ''.join(random.choice(letters) for i in range(length))
        self.fitness = -1
        
def snapshot(population):
    for i in range(10):
        print(population[i].ztext, population[i].fitness)

def initial_population():
    
    def new_member(word_length):
        return Member(word_length)
    
    population = []
    
    for i in range(population_size):
        newmember = new_member(word_length)
        population.append(newmember)
        
    return population


def fitness(population):
    for i in population:
        i.fitness = fuzz.ratio(i.ztext, target)
    sorted_population = sorted(population, key=lambda member: member.fitness, reverse=True)
    return sorted_population


def selection(population):
#     returns the fittest 20% of the population
    population = population[:int(0.2 * len(population))]
    return population


def crossover(population):
#     randomly selects two parents, randomly selects a crossover location, and then crosses
#     them and creates a new member from the resulting string. repeats until population
#     is back to desired size
    
    while len(population) < population_size:
    
        parent1 = random.choice(population).ztext
        parent2 = random.choice(population).ztext
        slicer = random.randint(0, word_length - 1)
        childtext = parent1[:slicer] + parent2[slicer:]

        new_member = Member(word_length)
        new_member.ztext = childtext

        population.append(new_member)
    
#         print(parent1, parent2, slicer, childtext)

    return population

def mutate(population):
    
    for member in population:
        if random.random() < 0.80:
            slicer = random.randint(0, word_length - 1)
            member.ztext = member.ztext[:slicer] + random.choice(string.ascii_lowercase) + member.ztext[slicer+1:]
    
    return population
        
    

In [230]:
zpop = initial_population()
zpop = fitness(zpop)

In [231]:
snapshot(zpop)

('ioppgai', 57)
('qkaggib', 43)
('beviyva', 43)
('ijmrilt', 43)
('coblqsc', 29)
('otaiwru', 29)
('hytubim', 29)
('nwahzpa', 29)
('gjgszay', 29)
('meqbarn', 29)


In [234]:
for i in range(10):
    zpop = selection(zpop)
    zpop = crossover(zpop)
    zpop = mutate(zpop)
    zpop = fitness(zpop)
    print(zpop[0].ztext)

abigail
abigail
abigail
abigail
abigail
abigail
abigail
abigail
abigail
abigail


In [235]:
snapshot(zpop)

('abigail', 100)
('abigail', 100)
('abigail', 100)
('abigail', 100)
('abigail', 100)
('abigail', 100)
('abigail', 100)
('abigail', 100)
('abigail', 100)
('abigail', 100)
